###  Pasos a seguir en el proceso de 'scraping':

1. Encuentra la URL que quieres 'escrapear'.
2. Inspecciona la página (código fuente).
3. Localiza los datos que necesitas obtener.
4. Desarrolla tu código en Python.
    1. Crea tu sopa
    2. Busca los elementos que cotienen los datos y extráelos
5. Ejecuta tu código y obten los datos.
6. Alamacena los datos en el formato requerido.

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np


## Caso 1: Scraping de un catálogo: Labirratorium

In [2]:
def clasifica_cervezas_pagina(cervezas_grid):
    '''Prepara todas las cervezas de una página de labirratorium.com y las clasifica en una lista de diccionarios'''
    lista_cervezas = []
    for cerveza in cervezas_grid:
        URL_cerveza = cerveza.find('a')['href']
        r = requests.get(URL_cerveza)
        if r.status_code == 200:
            soup_cerveza = bs(r.text, 'lxml')
            global current_index
            current_index += 1
            id_cerveza = f'lbt_{current_index}'
            try:
                name = soup_cerveza.find('h1').text
            except:
                name = '(sin nombre)'
            print(f'- Encontrada: {name} (id: {id_cerveza})')
            try:
                price = soup_cerveza.find(class_='current-price').text.split()[0].replace(',','.')
            except:
                price = None
                print('(sin precio)')
            try: 
                descr_short = "".join(soup_cerveza.find(class_='description-short').text.split('\n')).replace(u'\xa0', u' ')
            except:
                descr_short = None
                print('(sin descripcion corta)')
            try:
                descr_full = " ".join(soup_cerveza.find(class_='product-description').text.split())
            except:
                print('(sin descripción larga)')
            try:
                brand = soup_cerveza.find(class_='img img-thumbnail manufacturer-logo')['alt']
            except:
                brand = None
                print('(sin marca)')
            try:
                barcode = soup_cerveza.find(class_='product-reference').text.split()[1]
            except:
                barcode = None
                print('(sin código de barras)')
            try:
                image = soup_cerveza.find(class_='thumb js-thumb selected')['data-image-large-src']
            except:
                image = None
                print('(sin imagen)')
            try:
                features = {}
                features_mess = soup_cerveza.find(class_='data-sheet')
                categories = [category.string for category in features_mess.find_all('dt')]
                values = [value.string for value in features_mess.find_all('dd')]
                for category, value in zip(categories, values):
                    features[category] = value
            except:
                features = None
                print('(sin características)')
            cervezas_dict = {'id_cerveza': id_cerveza,
                                'name':name,
                                'price':price,
                                'descr_short': descr_short,
                                'descr_full': descr_full,
                                'brand': brand,
                                'barcode': barcode,
                                'image': image,
                                'features': features
                            }
            lista_cervezas.append(cervezas_dict)
        else:
            print(f'- Agotada: {cerveza.find("a")("img")[0]["alt"]} (id: {id_cerveza})')
    return lista_cervezas


print('En busca de cervezas:')
lista_maestra_cervezas = []
current_index = 0
for number in range(1, 31):
    URL = f'https://www.labirratorium.com/es/67-cervezas-por-estilo?page={number}'
    r = requests.get(URL)
    soup = bs(r.text, 'lxml')
    cervezas_grid = soup.find_all(class_="product-image")
    print(f'¡A por la página {number}! Llevamos {len(lista_maestra_cervezas)} cervezas.')
    lista_maestra_cervezas += clasifica_cervezas_pagina(cervezas_grid)

df_cervezas = pd.DataFrame(lista_maestra_cervezas)
df_cervezas.replace(r'^\s*$', np.nan, regex=True, inplace=True)

print('\nTabla de cervezas disponibles preparada.')


En busca de cervezas:
¡A por la página 1! Llevamos 0 cervezas.
- Encontrada: SanFrutos Vikingathor Boris Brew (id: lbt_1)
- Encontrada: Boon Oude Kriek 37,5cl (id: lbt_2)
- Encontrada: Orval (id: lbt_3)
- Encontrada: Schneider Aventinus Weizen-Eisbock (id: lbt_4)
- Encontrada: Aecht Schlenkerla Rauchbier Weizen (id: lbt_5)
- Encontrada: Samuel Adams Boston Lager (id: lbt_6)
- Encontrada: Westmalle Dubbel (id: lbt_7)
- Encontrada: Tripel Karmeliet 33 cl. (id: lbt_8)
- Encontrada: Weihenstephaner Vitus (id: lbt_9)
- Encontrada: Anchor Porter (id: lbt_10)
- Encontrada: Oud Beersel Oude Kriek Vieille 75 cl. (id: lbt_11)
- Encontrada: Weihenstephaner Korbinian (id: lbt_12)
¡A por la página 2! Llevamos 12 cervezas.
- Encontrada: Aecht Schlenkerla Rauchbier Märzen (id: lbt_13)
- Encontrada: Samuel Smith Imperial Stout (id: lbt_14)
- Encontrada: Westmalle Tripel (id: lbt_15)
- Encontrada: Chimay Azul Grande Réserve 75cl (id: lbt_16)
- Encontrada: Achel Brune (id: lbt_17)
- Encontrada: Ayinger 

In [3]:
df_cervezas

,id_cerveza,name,price,descr_short,descr_full,brand,barcode,image,features
0,lbt_1,SanFrutos Vikingathor Boris Brew,3.35,Indian Dunkel,"Cerveza de 8.2% ABV intensa, oscura y muy lupu...",Cerveza Sanfrutos,831,https://www.labirratorium.com/27941-large_defa...,"{'Estilo': 'DUNKEL', 'Origen': 'Segovia', '% A..."
1,lbt_2,"Boon Oude Kriek 37,5cl",7.15,Lambic / Kriek,Cerveza de fermentación espontánea (Lambic) de...,Brouwerij F. Boon,284,https://www.labirratorium.com/19351-large_defa...,"{'Estilo': 'KRIEK', 'Origen': 'Bélgica', '% Al..."
2,lbt_3,Orval,2.50,Belgian Pale Ale,Orval es una cerveza tipo ale y es la que prin...,Orval,199,https://www.labirratorium.com/385-large_defaul...,"{'Estilo': 'BELGIAN PALE ALE', 'Origen': 'Bélg..."
3,lbt_4,Schneider Aventinus Weizen-Eisbock,2.85,"Cerveza de color medio oscuro, aromas alcohóli...","Cerveza de color medio oscuro, aromas alcohóli...",Schneider,183,https://www.labirratorium.com/366-large_defaul...,"{'Origen': 'Alemania', '% Alc. Exacto': '12', ..."
4,lbt_5,Aecht Schlenkerla Rauchbier Weizen,2.70,NaN,Rauchbier (cerveza ahumada) de trigo con 5.2% ...,Schlenkerla,181,https://www.labirratorium.com/364-large_defaul...,"{'Estilo': 'RAUCHBIER', 'Origen': 'Alemania', ..."
...,...,...,...,...,...,...,...,...,...
352,lbt_353,MONTSENY LUPULUS,2.00,"Blonde Ale 5,4%","Blonde Ale 5,4% 29 IBU con lúpulos Cascade, fu...",None,19748,https://www.labirratorium.com/33088-large_defa...,"{'Estilo': 'BLOND ALE', '% Alc.': 'BAJO (2.5-4..."
353,lbt_354,MONTSENY ECO,2.60,Blonde Ale 5.4% ABV.29 IBU. 9 EBC,Blonde Ale 5.4% ABV. 29 IBU. 9 EBC Con malta P...,None,19750,https://www.labirratorium.com/33090-large_defa...,"{'Estilo': 'BLOND ALE', '% Alc.': 'BAJO (2.5-4..."
354,lbt_355,St. Louis Premium Gueuze,1.90,Gueuze 4.5% ABV,"Madurada en barrica de roble, esta lambic belg...",None,19821,https://www.labirratorium.com/33266-large_defa...,"{'Estilo': 'GUEUZE', 'Origen': 'Bélgica', '% A..."
355,lbt_356,Garage / Hudson Valley Clay Life MASH 22,4.90,"Extra Special Bitter 5,2% Lúpulo East Kent Gol...","Extra Special Bitter 5,2% Lúpulo East Kent Gol...",Garage Beer Co.,19836,https://www.labirratorium.com/33288-large_defa...,"{'Estilo': 'ENGLISH BITTER', '% Alc.': 'MEDIO ..."
